To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
</div>

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel

# Importar el modelo desde huggingface
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ljcamargo/tachiwin_translate",
    max_seq_length = 2048,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.10.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
)
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
tests_es = [
    "Lenguaje",
    "Casa",
    "Pueblo",
    "Gobierno de la República",
    "¿Cómo te llamas?",
    "¿Cuántos años tienes?",
    "¿Dónde está la escuela más cercana?",
    "¿vamos a comprar a la tienda?",
    "vamos a comer carne de puerco",
    "vamos al pueblo",
    "¿Puedes enseñarme a hablar Totonaco?",
    "La montaña es muy alta.",
    "voy a visitar a mis padres",
    "tengo mucho sueño",
    "Quiero conocer nuevos pueblos.",
    "quisiera viajar a la luna",
    "voy a comer un rico tamal de frijol",
    "¿Dónde vives?",
    "Estoy aprendiendo Totonaco.",
    "¿Qué comida te gusta más?",
    "Hoy es un día muy caluroso.",
    "¿Cómo puedo llegar al centro del pueblo?",
    "El río está lleno de peces.",
    "Me encanta bailar bajo la lluvia",
    "¿Has visto las estrellas esta noche?",
    "Mi abuela hace la mejor sopa del mundo",
    "El festival de música comienza mañana",
    "Necesito comprar semillas para mi jardín",
    "Los niños están jugando en el parque",
    "¿Podemos ver el atardecer juntos?",
    "Este café huele delicioso",
    "La biblioteca cierra a las ocho",
    "¿Cuál es tu temporada favorita del año?"
]

tests_en = [
    "Language",
    "House",
    "Town/Village",
    "Government of the Republic",
    "What is your name?",
    "How old are you?",
    "Where is the nearest school?",
    "Shall we go shopping at the store?",
    "Let's go eat pork",
    "Let's go to town",
    "Can you teach me to speak Totonac?",
    "The mountain is very high.",
    "I'm going to visit my parents",
    "I'm very sleepy",
    "I want to know new towns.",
    "I would like to travel to the moon",
    "I'm going to eat a delicious bean tamale",
    "Where do you live?",
    "I am learning Totonac.",
    "What food do you like the most?",
    "Today is a very hot day.",
    "How can I get to the town center?",
    "The river is full of fish.",
    "I love dancing in the rain",
    "Have you seen the stars tonight?",
    "My grandmother makes the best soup in the world",
    "The music festival starts tomorrow",
    "I need to buy seeds for my garden",
    "The children are playing in the park",
    "Can we watch the sunset together?",
    "This coffee smells delicious",
    "The library closes at eight",
    "What's your favorite season of the year?"
]

tests_to = [
    "¿tucha wanikana?",
    "¿nicha min akglit kga´ta?",
    "¿nicha wi xa lakgatsuw min pukgalhtawakga?",
    "kgáw tamawananaw kg´pustan?",
    "kgawaw xa lhiway paxni.",
    "kgaw kachikin",
    "¿kimasiyani nak´chiwinanxa Totonaco?",
    "wa kgestin lu xa lhankga",
    "kama lhakgapaxyalhnan ki natlhatna",
    "kg´lhtatatputun",
    "kg´lakgapasnamputun xa sasti kg´kachikinin",
    "kamputun kgosa anta niku wi pap´pa",
    "nak way makg´tam pulhakg´lha",
    "nicha xalha wix?",
    "kgatsinima nak chiwinan xa totonaco?",
    "nichama puwa tlhakg lhatakgatiya?",
    "uku lu xa kga´chichi",
    "lalha tlhan nak chan kg´xkgilhpuyatat kachikin?",
    "anta kg´kgalhtuchiokgo lhan wi skiti"
]


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

for text in tests_en:
  inputs = tokenizer(
  [
    alpaca_prompt.format(
        "Translate from (eng) to (tos)", # instruction
        text, # input
        "", # output - leave this blank for generation!
    )
  ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

for text in tests_to:
  inputs = tokenizer(
  [
    alpaca_prompt.format(
        "Translate from (tos) to (spa)", # instruction
        text, # input
        "", # output - leave this blank for generation!
    )
  ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


for text in tests_to:
  inputs = tokenizer(
  [
    alpaca_prompt.format(
        "Translate from (tos) to (eng)", # instruction
        text, # input
        "", # output - leave this blank for generation!
    )
  ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


for text in tests_es:
  inputs = tokenizer(
  [
    alpaca_prompt.format(
        "Translate from (spa) to (tos)", # instruction
        text, # input
        "", # output - leave this blank for generation!
    )
  ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Traducir del totonaco al español

### Input:
¿tucha wanikana?

### Response:
¿Qué haces?<|end_of_text|>
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Traducir del totonaco al español

### Input:
¿nicha min akglit kga´ta?

### Response:
¿Qué haces tú?<|end_of_text|>
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Traducir del totonaco al español

### Input:
¿nicha wi xa lakgatsuw min pukgalhtawakga?

### Response:
¿Dónde está tu padre?<|end_of_text|>
<|begin_of_text|>Below is an instruction that describes a tas

In [ ]:
text = "cómo está tu gato?"
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Translate from (spa) to (tos)", # instruction
        text, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Traducir del español al totonaco

### Input:
cómo está tu gato?

### Response:
lanlha wilha min paxni<|end_of_text|>


In [ ]:
text = "ni chan lha?"
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Translate from (tos) to (spa)", # instruction
        text, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [ ]:
text = "how do you do?"
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Translate from (eng) to (tos)", # instruction
        text, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)